In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
from random import shuffle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import activations
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

In [ ]:
GOOD_POSE_DIR = 'good_pose'
BAD_POSE_DIR = 'bad_pose'
IMG_SIZE = 256

In [ ]:
def label_image(image):
    """
    Function to encode and give a label to an image.
    image : (String) -> represents the image name.
    return : (Int) -> 0/1 encoding for a bad/good pose image.
    """
    world_label = image.split('_')[0]
    if world_label == 'good':
        return 1
    return 0

In [66]:
def process_data(form):
    """
    Function to process the data and save it in a numpy array form. (friendlier to neural networks)
    form : (String) -> represents the type of images it receives
    """
    data = []
    if form == 'good':
        dir = os.listdir(GOOD_POSE_DIR)
    else:
        dir = os.listdir(BAD_POSE_DIR)

    for image in dir:
        label = label_image(image)
        if form == 'good':
            path = os.path.join(GOOD_POSE_DIR, image)
        else:
            path = os.path.join(BAD_POSE_DIR, image)
        img = cv2.resize(cv2.imread(path, cv2.IMREAD_COLOR), (IMG_SIZE, IMG_SIZE))
        data.append([np.array(img), label])
    
    if form == 'good':
        np.save('good_pose_data.npy', data)
    else:
        np.save('bad_pose_data.npy', data)

In [67]:
# Loading the data
good_pose = np.load('good_pose_data.npy', allow_pickle=True)
bad_pose = np.load('bad_pose_data.npy', allow_pickle=True)

FileNotFoundError: [Errno 2] No such file or directory: 'good_pose_data.npy'

In [68]:
# Mixing the good pose data and bad pose data
good_and_bad_pose = np.concatenate((good_pose, bad_pose), axis=0)

# Split the data in train/test.
data = good_and_bad_pose[:, 0]
labels = good_and_bad_pose[:, 1]

X_train, X_valid, y_train, y_valid = train_test_split(data, labels, train_size=0.8)

y_train = np.asarray(y_train).astype(np.float32)
y_valid = np.asarray(y_valid).astype(np.float32)

## Running Openpose over the images to get the data

In [69]:
thr = 0.2
BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
               "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
               "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
               "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }

POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
               ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
               ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
               ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
               ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]

In [70]:
# Load the weigths
net = cv2.dnn.readNetFromTensorflow('graph_opt.pb')

error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-kh7iq4w7\opencv\modules\dnn\src\caffe\caffe_io.cpp:1133: error: (-2:Unspecified error) FAILED: fs.is_open(). Can't open "graph_opt.pb" in function 'cv::dnn::ReadProtoFromBinaryFile'


In [ ]:
def get_openpose_data(frame):
    """
    Function to run the openpose model over an image and get the coordonates for all 19 keypoints
    frame: (List) -> The image
    """
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]
    
    net.setInput(cv2.dnn.blobFromImage(frame, 1.0, (IMG_SIZE, IMG_SIZE), (127.5, 127.5, 127.5), swapRB=True, crop=False))
    out = net.forward()
    out = out[:, :19, :, :]

    assert(len(BODY_PARTS) == out.shape[1])

    points = []
    for i in range(len(BODY_PARTS)):
        heatMap = out[0, i, :, :]

        _, conf, _, point = cv2.minMaxLoc(heatMap)
        x = (frameWidth * point[0]) / out.shape[3]
        y = (frameHeight * point[1]) / out.shape[2]

        points.append((int(x), int(y)) if conf > thr else None)

    return points

In [61]:
X_train_openpose = []
X_valid_openpose = []

for image in X_train:
    X_train_openpose.append(get_openpose_data(image))

for image in X_valid:
    X_valid_openpose.append(get_openpose_data(image))

X_train_openpose = np.array(X_train_openpose)
X_valid_openpose = np.array(X_valid_openpose)

<ipython-input-61-c21555c44926>:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X_train_openpose = np.array(X_train_openpose)
<ipython-input-61-c21555c44926>:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  X_valid_openpose = np.array(X_valid_openpose)


In [ ]:
# Visualizing the keypoints
for coords in X_train_openpose:
    for coord, body_part in zip(coords, BODY_PARTS.keys()):
        if coord != None:
            print(f'{body_part} -> {coord}')
    print('\n')

## The fully conected model that takes the keypoint coordinates

In [62]:
model = Sequential()
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='relu'))
model.compile(loss='binary_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])

In [63]:
model.fit(X_train_openpose, y_train, epochs=100, batch_size=10)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type NoneType).

In [ ]:
output = model.predict(X_valid_openpose)

In [ ]:
print(np.round(output))

In [ ]:
print(confusion_matrix(y_valid, np.round(output)))
print(accuracy_score(y_valid, np.round(output)))

## Working on the video preprocessing

In [56]:
def fragment_video(video_name, save_location, interval):
    """
    video_name: String
    interval: Int -> interval in ms
    """
    vidcap = cv2.VideoCapture(video_name)
    success, image = vidcap.read()

    # Change the current directory.
    os.chdir(save_location)

    # While we have frames, we read and save them.
    count = 1
    while success:
        vidcap.set(cv2.CAP_PROP_POS_MSEC, (count*interval))
        cv2.imwrite(f'frame{count}.jpg', image)
        success, image = vidcap.read()
        count += 1

In [57]:
fragment_video('D:\College\coding (Python)\PoseNet - Biceps Curl\Ride_Along_2___Sneaking_Past_Security___Film_Clip.mp4', 'D:\College\coding (Python)\PoseNet - Biceps Curl\\test')